In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import streamlit as st

# Load data
matches = pd.read_csv('matches.csv')
deliveries = pd.read_csv('deliveries.csv')

# Streamlit App
st.title("IPL Data Analysis Dashboard")
st.markdown("Analyze team and player performances, toss/venue impact, and match trends.")

# Sidebar filters
season_filter = st.sidebar.selectbox("Select Season", sorted(matches['season'].unique()), index=0)
team_filter = st.sidebar.selectbox("Select Team", sorted(matches['team1'].unique()), index=0)

# Filter data based on season
season_df = matches[matches['season'] == season_filter]

# 1. Team Performance Analysis
st.subheader(f"Team Performance in {season_filter}")
team_matches = season_df[(season_df['team1'] == team_filter) | (season_df['team2'] == team_filter)]
team_wins = team_matches[team_matches['winner'] == team_filter].shape[0]
st.write(f"Matches Played: {team_matches.shape[0]} | Wins: {team_wins}")

# Win trends by season
season_wins = matches[matches['winner'] == team_filter].groupby('season').count()['id']
st.bar_chart(season_wins)

# 2. Player Stats
st.subheader("Top Batsmen by Runs")
top_batsmen = deliveries.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(10).reset_index()
st.dataframe(top_batsmen)

# 3. Venue & Toss Impact
st.subheader("Toss Impact on Match Outcome")
matches['toss_win_match_win'] = matches['toss_winner'] == matches['winner']
toss_impact = matches.groupby('season')['toss_win_match_win'].mean().reset_index()
fig = px.line(toss_impact, x='season', y='toss_win_match_win', title='Toss Win Leading to Match Win Ratio')
st.plotly_chart(fig)

# 4. Head-to-Head Comparison
st.subheader("Head-to-Head Stats")
opponent_filter = st.selectbox("Select Opponent Team", sorted(matches['team2'].unique()))
head_to_head = matches[((matches['team1'] == team_filter) & (matches['team2'] == opponent_filter)) | ((matches['team2'] == team_filter) & (matches['team1'] == opponent_filter))]
h2h_wins = head_to_head['winner'].value_counts().reset_index()
st.dataframe(h2h_wins.rename(columns={'index': 'Team', 'winner': 'Wins'}))

# 5. Win Prediction (Basic Logistic Regression Example)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

st.subheader("Win Prediction Model (Simple Demo)")
model_data = matches.dropna(subset=['winner'])
le = LabelEncoder()
model_data['team1_enc'] = le.fit_transform(model_data['team1'])
model_data['team2_enc'] = le.fit_transform(model_data['team2'])
model_data['toss_winner_enc'] = le.fit_transform(model_data['toss_winner'])
model_data['venue_enc'] = le.fit_transform(model_data['venue'])
model_data['winner_enc'] = le.fit_transform(model_data['winner'])

X = model_data[['team1_enc', 'team2_enc', 'toss_winner_enc', 'venue_enc']]
y = model_data['winner_enc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
st.write(f"Model Accuracy: {model.score(X_test, y_test):.2f}")

# 6. Dynamic Visuals
st.subheader("Season-wise Match Results")
match_counts = matches.groupby('season')['id'].count().reset_index(name='Total Matches')
fig2 = px.bar(match_counts, x='season', y='Total Matches', title='Matches Played Per Season')
st.plotly_chart(fig2)

st.markdown("---")
st.caption("Built with ❤️ using Streamlit and IPL data")